In [2]:
import numpy as np
import pandas as pd 
import math
from simpledbf import Dbf5 

In [3]:
data0 = "C:\\Users\\Research\\Documents\\Tara_Fall_2019\\dosso_index\\base_HWSD_slope0.dbf"
data69 = "C:\\Users\\Research\\Documents\\Tara_Fall_2019\\dosso_index\\base_HWSD_slope69.dbf"
data116 = "C:\\Users\\Research\\Documents\\Tara_Fall_2019\\dosso_index\\base_HWSD_slope116.dbf"
join_data = "C:\\Users\\Research\\Documents\\Tara_Fall_2019\\dosso_index\\HWSd_dosso_la.dbf"
dbf0 = Dbf5(data0)
dbf69 = Dbf5(data69)
dbf116 = Dbf5(data116)
join_dbf = Dbf5(join_data)
df_data0 = dbf0.to_dataframe()
df_data69 = dbf69.to_dataframe()
df_data116 = dbf116.to_dataframe()
df_join = join_dbf.to_dataframe()

In [4]:
df0 = df_data0.merge(df_join, on = 'OBJECTID', how = "left")
df69 = df_data69.merge(df_join, on = 'OBJECTID', how = "left")
df116 = df_data116.merge(df_join, on = 'OBJECTID', how = "left")

Prep code for the raw data files

In [5]:
# function to assign USDA texture class in string format to numeric classes from HWSD dataset 
# apply to T_USDA_TEX column and S_USDA_TEX column
def inttex_to_stringtex(USDA_tex):
    switcher = {
        1: "clay",
        2: "silty clay",
        3: "clay",
        4: "silty clay loam",
        5: "clay loam",
        6: "silt",
        7: "silt loam",
        8: "sandy clay",
        9: "loam",
        10: "sandy clay loam",
        11: "sandy loam",
        12: "loamy sand",
        13: "sand"
    }
    return switcher.get(USDA_tex, "err")

In [6]:
df0['T_USDA_TEX'] = df0['T_USDA_TEX'].apply(inttex_to_stringtex)
df0['S_USDA_TEX'] = df0['S_USDA_TEX'].apply(inttex_to_stringtex)
df69['T_USDA_TEX'] = df69['T_USDA_TEX'].apply(inttex_to_stringtex)
df69['S_USDA_TEX'] = df69['S_USDA_TEX'].apply(inttex_to_stringtex)
df116['T_USDA_TEX'] = df116['T_USDA_TEX'].apply(inttex_to_stringtex)
df116['S_USDA_TEX'] = df116['S_USDA_TEX'].apply(inttex_to_stringtex)

In [7]:
# changing columns to be used in calculations
df0['slope']=df0['GRIDCODE']/10000

df69['slope']=df69['GRIDCODE']/10000

df116['slope']=df116['GRIDCODE']/10000

Functions to be nested into final LCC calculation function

In [147]:
# function to calculate erosion risk LCC class
# calculates K factor and combines with slope to determine risk
def erosion_risk_LCC_T(row):
    texture = row["T_USDA_TEX"]
    slope = row['slope']
    #first calculate k factor based on texture
    list1 = ["loam", "silt loam", "silt"]
    if list1.count(texture) > 0:
        k_fac = .33
    else: 
        k_fac = .31
    
    #then calculate lcc class of erosion risk combining k factor and slope
    if k_fac >.32 and slope <=2:
        erosion_risk_lcc = 1
    elif k_fac >.32 and (slope > 2 and slope <=5):
        erosion_risk_lcc = 2
    elif k_fac >.32 and (slope > 5 and slope <= 10):
        erosion_risk_lcc = 3
    elif k_fac >.32 and (slope > 10 and slope <= 15):
        erosion_risk_lcc = 4
    elif k_fac >.32 and (slope > 15 and slope <= 30):
        erosion_risk_lcc = 6    
    elif k_fac >.32 and (slope > 30 and slope <= 60):
        erosion_risk_lcc = 7    
    elif k_fac >.32 and slope > 60:
        erosion_risk_lcc = 8
    elif k_fac <= 0.32 and slope <= 5:
        erosion_risk_lcc = 1       
    elif k_fac <= 0.32 and (slope > 5 and slope <= 10):
        erosion_risk_lcc = 2 
    elif k_fac <= 0.32 and (slope > 10 and slope <= 15):
        erosion_risk_lcc = 3
    elif k_fac <= 0.32 and (slope > 15 and slope <= 30):
        erosion_risk_lcc = 4   
    elif k_fac <= 0.32 and (slope > 30 and slope <= 60):
        erosion_risk_lcc = 7    
    elif k_fac <= 0.32 and slope > 60:
        erosion_risk_lcc = 8
    else: 
        erosion_risk_lcc = 'err'
    
    return erosion_risk_lcc

def erosion_risk_LCC_S(row):
    texture = row["S_USDA_TEX"]
    slope = row['slope']
    #first calculate k factor based on texture
    list1 = ["loam", "silt loam", "silt"]
    if list1.count(texture) > 0:
        k_fac = .33
    else: 
        k_fac = .31
    
    #then calculate lcc class of erosion risk combining k factor and slope
    if k_fac >.32 and slope <=2:
        erosion_risk_lcc = 1
    elif k_fac >.32 and (slope > 2 and slope <=5):
        erosion_risk_lcc = 2
    elif k_fac >.32 and (slope > 5 and slope <= 10):
        erosion_risk_lcc = 3
    elif k_fac >.32 and (slope > 10 and slope <= 15):
        erosion_risk_lcc = 4
    elif k_fac >.32 and (slope > 15 and slope <= 30):
        erosion_risk_lcc = 6    
    elif k_fac >.32 and (slope > 30 and slope <= 60):
        erosion_risk_lcc = 7    
    elif k_fac >.32 and slope > 60:
        erosion_risk_lcc = 8
    elif k_fac <= 0.32 and slope <= 5:
        erosion_risk_lcc = 1       
    elif k_fac <= 0.32 and (slope > 5 and slope <= 10):
        erosion_risk_lcc = 2 
    elif k_fac <= 0.32 and (slope > 10 and slope <= 15):
        erosion_risk_lcc = 3
    elif k_fac <= 0.32 and (slope > 15 and slope <= 30):
        erosion_risk_lcc = 4   
    elif k_fac <= 0.32 and (slope > 30 and slope <= 60):
        erosion_risk_lcc = 7    
    elif k_fac <= 0.32 and slope > 60:
        erosion_risk_lcc = 8
    else: 
        erosion_risk_lcc = 'err'
    
    return erosion_risk_lcc

In [148]:
# function to calculate surface soil texture LCC class
# takes in texture and outputs LCC class
def soil_tex_LCC_T(row):
    texture = row["T_USDA_TEX"]
    list1 = ["sandy loam", "silt loam", "loam", "silt", "sandy clay loam", "silty clay loam", "clay loam"]
    list2 = ["sand", "loamy sand", "sandy clay", "silty clay"] 
    
    if list1.count(texture) > 0 :
        surface_st_lcc = 1
    elif list2.count(texture) > 0 :
        surface_st_lcc = 2
    elif texture == "clay":
        surface_st_lcc = 3
    else:
        surface_st_lcc = "err"
    
    return surface_st_lcc

def soil_tex_LCC_S(row):
    texture = row["S_USDA_TEX"]
    list1 = ["sandy loam", "silt loam", "loam", "silt", "sandy clay loam", "silty clay loam", "clay loam"]
    list2 = ["sand", "loamy sand", "sandy clay", "silty clay"] 
    
    if list1.count(texture) > 0 :
        surface_st_lcc = 1
    elif list2.count(texture) > 0 :
        surface_st_lcc = 2
    elif texture == "clay":
        surface_st_lcc = 3
    else:
        surface_st_lcc = "err"
    
    return surface_st_lcc

In [149]:
# function to calculate available water capacity LCC class
# takes in awc class and outputs LCC class
# USES AWC FROM HWSD RATHER THAN CALCULATING AWC IN THE LANDPKS MANNER
def awc_LCC(row):
    awc_class = row["AWC_CLASS"]
    if awc_class in [1, 2]:
        awc_lcc = 2
    elif awc_class in [3, 4]:
        awc_lcc = 3
    elif awc_class in [5, 6, 7]:
        awc_lcc = 4
    else:
        awc_lcc = "err"
    
    return awc_lcc

In [150]:
# function to calculate available water capacity LCC class
# takes in awc class and outputs LCC class
# USES AWC FROM HWSD RATHER THAN CALCULATING AWC IN THE LANDPKS MANNER
def awc(AWC_CLASS):
    switcher = {
        1: .150,
        2: .125,
        3: .100,
        4: .075,
        5: .050,
        6: .015,
        7: 0
    }
    return switcher.get(AWC_CLASS, "err")

In [151]:
#calculate AWC values for topsoil and subsoil based on HWSD discrete classes
df0['T_AWC'] = (df0['AWC_CLASS'].apply(awc))*30
df0['S_AWC'] = (df0['AWC_CLASS'].apply(awc))*70
df69['T_AWC'] = (df69['AWC_CLASS'].apply(awc))*30
df69['S_AWC'] = (df69['AWC_CLASS'].apply(awc))*70
df116['T_AWC'] = (df116['AWC_CLASS'].apply(awc))*30
df116['S_AWC'] = (df116['AWC_CLASS'].apply(awc))*70

Functions for components of LCC calculation that are currently not available with our data

In [153]:
#  function to calculate soil depth LCC class
#  takes in phases and outputs LCC class if lithic phase is present 
def depth_LCC(row):
    phase1 = row["PHASE1"]
    phase2 = row["PHASE2"]
    if phase1 == 2 or phase2 == 2:
        depth_LCC = 6
    else: 
        depth_LCC = 1
    
    return depth_LCC

In [154]:
# function to calculate salinity LCC class
# takes in T_ECE and outputs LCC class 
def salinity_LCC_T(row):
    t_ece = row['T_ECE']
    if t_ece < 2:
        salinity_lcc = 1
    elif t_ece >= 2 and t_ece <= 4: 
        salinity_lcc = 2
    elif t_ece > 4 and t_ece <= 8: 
        salinity_lcc = 3
    elif t_ece > 8 and t_ece <= 16: 
        salinity_lcc = 4
    elif t_ece > 16 and t_ece <= 30: 
        salinity_lcc = 6
    elif t_ece > 30 and t_ece <= 40: 
        salinity_lcc = 7
    elif t_ece > 40: 
        salinity_lcc = 8
    
    return salinity_lcc

def salinity_LCC_S(row):
    t_ece = row['S_ECE']
    if t_ece < 2:
        salinity_lcc = 1
    elif t_ece >= 2 and t_ece <= 4: 
        salinity_lcc = 2
    elif t_ece > 4 and t_ece <= 8: 
        salinity_lcc = 3
    elif t_ece > 8 and t_ece <= 16: 
        salinity_lcc = 4
    elif t_ece > 16 and t_ece <= 30: 
        salinity_lcc = 6
    elif t_ece > 30 and t_ece <= 40: 
        salinity_lcc = 7
    elif t_ece > 40: 
        salinity_lcc = 8
    
    return salinity_lcc

In [155]:
# function to calculate lime requirement LCC class
# takes in T_PH_H2O and outputs LCC class 
def lime_LCC_T(row):
    ph_val = row["T_PH_H2O"]
    if ph_val >= 5.5 and ph_val <= 7.2: 
        lime_lcc = 1
    elif (ph_val < 5.5 and ph_val >=4.5) or (ph_val > 7.2 and ph_val <= 8.4): 
        lime_lcc = 3
    elif ph_val < 4.5 or ph_val > 8.5:
        lime_lcc = 4
    
    return lime_lcc

def lime_LCC_S(row):
    ph_val = row["S_PH_H2O"]
    if ph_val >= 5.5 and ph_val <= 7.2: 
        lime_lcc = 1
    elif (ph_val < 5.5 and ph_val >=4.5) or (ph_val > 7.2 and ph_val <= 8.4): 
        lime_lcc = 3
    elif ph_val < 4.5 or ph_val > 8.5:
        lime_lcc = 4
    
    return lime_lcc

In [157]:
# function to calculate permeability LCC class
# take in texture, calculate intermediary values, outpus LCC class
# based on landPKs alculation of permeability
def perm_LCC_T(row):
    sand = row["T_SAND"]/100
    silt = row["T_SILT"]/100
    clay = row["T_CLAY"]/100
    om = 2.17*row["T_OC"]/100
    gravel = row["T_GRAVEL"]/100
    
    WP1 = -0.024*(sand) + 0.487*(clay) + 0.006*(om) + 0.005*(sand)*(om) - 0.013*(clay)*(om) + 0.068*(sand)*(clay) + 0.031
    WP = WP1 + (0.14*WP1 - 0.02)
    FC1 = -0.251*(sand) + 0.195*(clay) + 0.011*(om) + 0.006*(sand)*(om) - 0.027*(clay)*(om) + 0.452*(sand)*(clay) + 0.299
    FC = FC1 + (1.283*FC1*FC1 - 0.374*FC1 - 0.015)
    LAWC = ( FC - WP )*(1 - (gravel))
    SFC1 = 0.278*(sand) + 0.034*(clay) + 0.022*(om) - 0.018*(sand)*(om) - 0.027*(clay)*(om) - 0.584*(sand)*(clay) + 0.078
    SFC = SFC1 + ((0.636*SFC1) - 0.107)
    SASC = FC + SFC - 0.097*(sand) + 0.043
    MSD = (1- SASC)*2.65
    BSD = (1 - (gravel)) / (1 - (gravel)*(1 - 1.5*(MSD / 2.65)))
    L = (math.log(WP) - math.log(FC)) / (math.log(1500) - math.log(33))
    KS = 1930*(SASC)**(3-L)*BSD
    

    if KS < 1.5:
        perm_lcc = 3
    elif KS >= 1.5 and KS <= 5:
        perm_lcc = 2
    elif KS > 5:
        perm_lcc = 1
    else: 
        perm_lcc = 'err'
        
    return perm_lcc

def perm_LCC_S(row):
    sand = row["S_SAND"]/100
    silt = row["S_SILT"]/100
    clay = row["S_CLAY"]/100
    om = 2.17*row["S_OC"]/100
    gravel = row["S_GRAVEL"]/100
    
    WP1 = -0.024*(sand) + 0.487*(clay) + 0.006*(om) + 0.005*(sand)*(om) - 0.013*(clay)*(om) + 0.068*(sand)*(clay) + 0.031
    WP = WP1 + (0.14*WP1 - 0.02)
    FC1 = -0.251*(sand) + 0.195*(clay) + 0.011*(om) + 0.006*(sand)*(om) - 0.027*(clay)*(om) + 0.452*(sand)*(clay) + 0.299
    FC = FC1 + (1.283*FC1*FC1 - 0.374*FC1 - 0.015)
    LAWC = ( FC - WP )*(1 - (gravel))
    SFC1 = 0.278*(sand) + 0.034*(clay) + 0.022*(om) - 0.018*(sand)*(om) - 0.027*(clay)*(om) - 0.584*(sand)*(clay) + 0.078
    SFC = SFC1 + ((0.636*SFC1) - 0.107)
    SASC = FC + SFC - 0.097*(sand) + 0.043
    MSD = (1- SASC)*2.65
    BSD = (1 - (gravel)) / (1 - (gravel)*(1 - 1.5*(MSD / 2.65)))
    L = (math.log(WP) - math.log(FC)) / (math.log(1500) - math.log(33))
    KS = 1930*(SASC)**(3-L)*BSD
    

    if KS < 1.5:
        perm_lcc = 3
    elif KS >= 1.5 and KS <= 5:
        perm_lcc = 2
    elif KS > 5:
        perm_lcc = 1
    else: 
        perm_lcc = 'err'
        
    return perm_lcc


In [158]:
# function to calculate permeability LCC class
# take in texture, calculate intermediary values, outpus LCC class
# based on landPKs alculation of permeability
def LAWC_T(row):
    sand = row["T_SAND"]/100
    silt = row["T_SILT"]/100
    clay = row["T_CLAY"]/100
    om = 2.17*row["T_OC"]/100
    gravel = row["T_GRAVEL"]/100
    
    WP1 = -0.024*(sand) + 0.487*(clay) + 0.006*(om) + 0.005*(sand)*(om) - 0.013*(clay)*(om) + 0.068*(sand)*(clay) + 0.031
    WP = WP1 + (0.14*WP1 - 0.02)
    FC1 = -0.251*(sand) + 0.195*(clay) + 0.011*(om) + 0.006*(sand)*(om) - 0.027*(clay)*(om) + 0.452*(sand)*(clay) + 0.299
    FC = FC1 + (1.283*FC1*FC1 - 0.374*FC1 - 0.015)
    LAWC = ( FC - WP )*(1 - (gravel))
    # MULTIPLY BY DEPTH OF SOIL HERE
    AWC = LAWC*30
#     if AWC > 18:
#         awc_lcc = 1
#     elif AWC > 12 and LAWC <= 18:
#         awc_lcc = 2
#     elif AWC > 6 and LAWC <= 12:
#         awc_lcc = 3
#     elif AWC <= 6: 
#         awc_lcc = 4
#     else:
#         awc_lcc = 0
        
    return AWC

def LAWC_S(row):
    sand = row["S_SAND"]/100
    silt = row["S_SILT"]/100
    clay = row["S_CLAY"]/100
    om = 2.17*row["S_OC"]/100
    gravel = row["S_GRAVEL"]/100
    
    WP1 = -0.024*(sand) + 0.487*(clay) + 0.006*(om) + 0.005*(sand)*(om) - 0.013*(clay)*(om) + 0.068*(sand)*(clay) + 0.031
    WP = WP1 + (0.14*WP1 - 0.02)
    FC1 = -0.251*(sand) + 0.195*(clay) + 0.011*(om) + 0.006*(sand)*(om) - 0.027*(clay)*(om) + 0.452*(sand)*(clay) + 0.299
    FC = FC1 + (1.283*FC1*FC1 - 0.374*FC1 - 0.015)
    LAWC = ( FC - WP )*(1 - (gravel))
    AWC = LAWC*70
#     if AWC > 18:
#         awc_lcc = 1
#     elif AWC > 12 and LAWC <= 18:
#         awc_lcc = 2
#     elif AWC > 6 and LAWC <= 12:
#         awc_lcc = 3
#     elif AWC <= 6: 
#         awc_lcc = 4
#     else:
#         awc_lcc = 0
        
    return AWC

In [159]:
def LAWC_LCC(row):
    Tawc = row["T_LAWC"]
    Sawc = row["S_LAWC"]
    
    AWC = Tawc + Sawc
    if AWC > 18:
        awc_lcc = 1
    elif AWC > 12 and AWC <= 18:
        awc_lcc = 2
    elif AWC > 6 and AWC <= 12:
        awc_lcc = 3
    elif AWC <= 6: 
        awc_lcc = 4
    else:
        awc_lcc = 0
    
    return awc_lcc

Final LCC Calculation function

In [160]:
df0["T_erosion_risk_LCC"] = df0.apply(erosion_risk_LCC_T, axis=1)
df0["T_soil_tex_LCC"] = df0.apply(soil_tex_LCC_T, axis=1)
df0["Total_awc_LCC"] = df0.apply(awc_LCC, axis=1)
df0["depth_LCC"] = df0.apply(depth_LCC, axis=1)
df0["T_salinity_LCC"] = df0.apply(salinity_LCC_T, axis=1)
df0["T_lime_LCC"] = df0.apply(lime_LCC_T, axis=1)
df0["wtd_LCC"] = df0.apply(wtd_LCC, axis=1)
df0["T_perm_LCC"] = df0.apply(perm_LCC_T, axis=1)
df0["T_LAWC"] = df0.apply(LAWC_T, axis=1)

df0["S_erosion_risk_LCC"] = df0.apply(erosion_risk_LCC_S, axis=1)
df0["S_soil_tex_LCC"] = df0.apply(soil_tex_LCC_S, axis=1)
df0["S_salinity_LCC"] = df0.apply(salinity_LCC_S, axis=1)
df0["S_lime_LCC"] = df0.apply(lime_LCC_S, axis=1)
df0["S_LAWC"] = df0.apply(LAWC_S, axis=1)
df0["S_perm_LCC"] = df0.apply(perm_LCC_S, axis=1)
df0["Total_LAWC_LCC"] = df0.apply(LAWC_LCC, axis=1)

df69["T_erosion_risk_LCC"] = df69.apply(erosion_risk_LCC_T, axis=1)
df69["T_soil_tex_LCC"] = df69.apply(soil_tex_LCC_T, axis=1)
df69["Total_awc_LCC"] = df69.apply(awc_LCC, axis=1)
df69["depth_LCC"] = df69.apply(depth_LCC, axis=1)
df69["T_salinity_LCC"] = df69.apply(salinity_LCC_T, axis=1)
df69["T_lime_LCC"] = df69.apply(lime_LCC_T, axis=1)
df69["wtd_LCC"] = df69.apply(wtd_LCC, axis=1)
df69["T_perm_LCC"] = df69.apply(perm_LCC_T, axis=1)
df69["T_LAWC"] = df69.apply(LAWC_T, axis=1)

df69["S_erosion_risk_LCC"] = df69.apply(erosion_risk_LCC_S, axis=1)
df69["S_soil_tex_LCC"] = df69.apply(soil_tex_LCC_S, axis=1)
df69["S_salinity_LCC"] = df69.apply(salinity_LCC_S, axis=1)
df69["S_lime_LCC"] = df69.apply(lime_LCC_S, axis=1)
df69["S_perm_LCC"] = df69.apply(perm_LCC_S, axis=1)
df69["S_LAWC"] = df69.apply(LAWC_S, axis=1)
df69["Total_LAWC_LCC"] = df69.apply(LAWC_LCC, axis=1)

df116["T_erosion_risk_LCC"] = df116.apply(erosion_risk_LCC_T, axis=1)
df116["T_soil_tex_LCC"] = df116.apply(soil_tex_LCC_T, axis=1)
df116["Total_awc_LCC"] = df116.apply(awc_LCC, axis=1)
df116["depth_LCC"] = df116.apply(depth_LCC, axis=1)
df116["T_salinity_LCC"] = df116.apply(salinity_LCC_T, axis=1)
df116["T_lime_LCC"] = df116.apply(lime_LCC_T, axis=1)
df116["wtd_LCC"] = df116.apply(wtd_LCC, axis=1)
df116["T_perm_LCC"] = df116.apply(perm_LCC_T, axis=1)
df116["T_LAWC"] = df116.apply(LAWC_T, axis=1)

df116["S_erosion_risk_LCC"] = df116.apply(erosion_risk_LCC_S, axis=1)
df116["S_soil_tex_LCC"] = df116.apply(soil_tex_LCC_S, axis=1)
df116["S_salinity_LCC"] = df116.apply(salinity_LCC_S, axis=1)
df116["S_lime_LCC"] = df116.apply(lime_LCC_S, axis=1)
df116["S_perm_LCC"] = df116.apply(perm_LCC_S, axis=1)
df116["S_LAWC"] = df116.apply(LAWC_S, axis=1)
df116["Total_LAWC_LCC"] = df116.apply(LAWC_LCC, axis=1)

In [121]:
# HERE

In [163]:
df69["Total_LAWC_LCC"].unique()

array([4, 2, 3], dtype=int64)

In [164]:
# function to calculate overall LCC 
# input one row of the dataframe, to be applied to the full dataframe
def calc_LCC_T(row):
    lcc_final = max(row["T_erosion_risk_LCC"], row["T_soil_tex_LCC"], row["Total_awc_LCC"], row["depth_LCC"], 
                   row["T_salinity_LCC"], row["T_lime_LCC"], row["wtd_LCC"], row["T_perm_LCC"], row["Total_LAWC_LCC"])
    
    return lcc_final

def calc_LCC_S(row):
    lcc_final = max(row["S_erosion_risk_LCC"], row["S_soil_tex_LCC"], row["Total_awc_LCC"], row["depth_LCC"], 
                   row["S_salinity_LCC"], row["S_lime_LCC"], row["wtd_LCC"], row["Total_LAWC_LCC"] )
    
    return lcc_final

In [165]:
df0["LCC_final_T"] = df0.apply(calc_LCC_T, axis=1)
df0["LCC_final_S"] = df0.apply(calc_LCC_S, axis=1)
df69["LCC_final_T"] = df69.apply(calc_LCC_T, axis=1)
df69["LCC_final_S"] = df69.apply(calc_LCC_S, axis=1)
df116["LCC_final_T"] = df116.apply(calc_LCC_T, axis=1)
df116["LCC_final_S"] = df116.apply(calc_LCC_S, axis=1)

In [1]:
def limitation_T(row):
    LCC_final = row['LCC_final_T']
    erosion = row['T_erosion_risk_LCC']
    tex = row['T_soil_tex_LCC']
    perm = row['T_perm_LCC']
    awc = row['Total_awc_LCC']
    lime = row['T_lime_LCC']
    depth = row['depth_LCC']
    salinity = row['T_salinity_LCC']
    wtd = row['wtd_LCC']
    LAWC = row['Total_LAWC_LCC']
    
    lim = str(LCC_final)
    if erosion == LCC_final: 
        lim += "e"
    if tex == LCC_final: 
        lim +="s-t"
    if perm == LCC_final: 
        lim +="w-p"
    if awc == LCC_final: 
        lim +="s-a"
    if lime == LCC_final: 
        lim +="s-l"
    if depth == LCC_final: 
        lim +="s-d"
    if salinity == LCC_final: 
        lim +="s-k"
    if wtd == LCC_final: 
        lim +="w-d"
    if LAWC == LCC_final: 
        lim +="s-La"
    
    return(lim)

def limitation_S(row):
    LCC_final = row['LCC_final_S']
    erosion = row['S_erosion_risk_LCC']
    tex = row['S_soil_tex_LCC']
    perm = row['S_perm_LCC']
    awc = row['Total_awc_LCC']
    lime = row['S_lime_LCC']
    depth = row['depth_LCC']
    salinity = row['S_salinity_LCC']
    wtd = row['wtd_LCC']
    LAWC = row['Total_LAWC_LCC']
    
    lim = str(LCC_final)
    if erosion == LCC_final: 
        lim += "e"
    if tex == LCC_final: 
        lim +="s-t"
    if perm == LCC_final: 
        lim +="w-p"
    if awc == LCC_final: 
        lim +="s-a"
    if lime == LCC_final: 
        lim +="s-l"
    if depth == LCC_final: 
        lim +="s-d"
    if salinity == LCC_final: 
        lim +="s-k"
    if wtd == LCC_final: 
        lim +="w-d"
    if LAWC == LCC_final: 
        lim +="s-La"
    
    return(lim)

In [167]:
df0["T_limitation"] = df0.apply(limitation_T, axis=1)
df0["S_limitation"] = df0.apply(limitation_S, axis=1)
df69["T_limitation"] = df69.apply(limitation_T, axis=1)
df69["S_limitation"] = df69.apply(limitation_S, axis=1)
df116["T_limitation"] = df116.apply(limitation_T, axis=1)
df116["S_limitation"] = df116.apply(limitation_S, axis=1)

In [174]:
df0.columns[10:]

Index(['CompID', 'mukey', 'MU_GLOBAL', 'MU_SOURCE1', 'MU_SOURCE2', 'ISSOIL',
       'SHARE', 'SEQ', 'SU_SYM74', 'SU_CODE74', 'SU_SYM85', 'SU_CODE85',
       'SU_SYM90', 'SU_CODE90', 'T_TEXTURE', 'DRAINAGE', 'REF_DEPTH',
       'AWC_CLASS', 'PHASE1', 'PHASE2', 'ROOTS', 'IL', 'SWR', 'ADD_PROP',
       'T_GRAVEL', 'T_SAND', 'T_SILT', 'T_CLAY', 'T_USDA_TEX', 'T_REF_BULK',
       'T_BULK_DEN', 'T_OC', 'T_PH_H2O', 'T_CEC_CLAY', 'T_CEC_SOIL', 'T_BS',
       'T_TEB', 'T_CACO3', 'T_CASO4', 'T_ESP', 'T_ECE', 'S_GRAVEL', 'S_SAND',
       'S_SILT', 'S_CLAY', 'S_USDA_TEX', 'S_REF_BULK', 'S_BULK_DEN', 'S_OC',
       'S_PH_H2O', 'S_CEC_CLAY', 'S_CEC_SOIL', 'S_BS', 'S_TEB', 'S_CACO3',
       'S_CASO4', 'S_ESP', 'S_ECE', 'FAO_Year', 'FAO_SU', 'FAO_Value',
       'FAO74_CD', 'FAO74_SU', 'FAO90_CD', 'FAO90_SU', 'XLOG', 'YLAT',
       'ISO_CODE', 'CNTRY_NAME', 'Soil_ID', 'slope', 'T_AWC', 'S_AWC',
       'T_erosion_risk_LCC', 'T_soil_tex_LCC', 'Total_awc_LCC', 'depth_LCC',
       'T_salinity_LCC', 'T_lime

In [ ]:
'AWC_CLASS','T_GRAVEL', 'T_SAND', 'T_SILT', 'T_CLAY', 'T_OC', 'T_PH_H2O','T_ECE', 'S_GRAVEL', 'S_SAND',
'S_SILT', 'S_CLAY', 'S_OC','S_PH_H2O',  'S_ECE', 'slope', 'T_AWC', 'S_AWC','T_erosion_risk_LCC', 
'T_soil_tex_LCC', 'Total_awc_LCC', 'depth_LCC','T_salinity_LCC', 'T_lime_LCC', 'wtd_LCC', 'T_perm_LCC', 
'T_LAWC','S_erosion_risk_LCC', 'S_soil_tex_LCC', 'S_salinity_LCC', 'S_lime_LCC','S_LAWC', 'S_perm_LCC', 
'Total_LAWC_LCC', 'LCC_final_T', 'LCC_final_S','T_limitation', 'S_limitation'

In [57]:
df0.columns

,Join_Count,TARGET_FID,Join_Cou_1,TARGET_F_1,OBJECTID,ID,GRIDCODE,FID_1,Id,gridcode,...,S_erosion_risk_LCC,S_soil_tex_LCC,S_salinity_LCC,S_lime_LCC,S_perm_LCC,S_LAWC_LCC,LCC_final_T,LCC_final_S,limitation_T,limitation_S
0,1,0,6,0,267173,443009,25606,234815,267173,1618,...,1,2,1,1,1,3,3,3,awc_LCC,awc_LCC
1,1,1,3,1,267173,437316,0,234815,267173,1618,...,1,2,1,1,1,3,3,3,awc_LCC,awc_LCC
2,1,2,6,2,267173,423720,32376,234815,267173,1618,...,1,2,1,1,1,3,3,3,awc_LCC,awc_LCC
3,1,3,8,3,267173,328928,45739,234815,267173,1618,...,1,2,1,1,1,3,3,3,awc_LCC,awc_LCC
4,1,4,8,4,267173,434023,47140,234815,267173,1618,...,1,2,1,1,1,3,3,3,awc_LCC,awc_LCC


In [175]:
df0_succinct = df0[['AWC_CLASS','T_GRAVEL', 'T_SAND', 'T_SILT', 'T_CLAY', 'T_OC', 'T_PH_H2O','T_ECE', 'S_GRAVEL', 'S_SAND',
'S_SILT', 'S_CLAY', 'S_OC','S_PH_H2O',  'S_ECE', 'slope', 'T_AWC', 'S_AWC','T_erosion_risk_LCC', 
'T_soil_tex_LCC', 'Total_awc_LCC', 'depth_LCC','T_salinity_LCC', 'T_lime_LCC', 'wtd_LCC', 'T_perm_LCC', 
'T_LAWC','S_erosion_risk_LCC', 'S_soil_tex_LCC', 'S_salinity_LCC', 'S_lime_LCC','S_LAWC', 'S_perm_LCC', 
'Total_LAWC_LCC', 'LCC_final_T', 'LCC_final_S','T_limitation', 'S_limitation']].copy()
df69_succinct = df69[['AWC_CLASS','T_GRAVEL', 'T_SAND', 'T_SILT', 'T_CLAY', 'T_OC', 'T_PH_H2O','T_ECE', 'S_GRAVEL', 'S_SAND',
'S_SILT', 'S_CLAY', 'S_OC','S_PH_H2O',  'S_ECE', 'slope', 'T_AWC', 'S_AWC','T_erosion_risk_LCC', 
'T_soil_tex_LCC', 'Total_awc_LCC', 'depth_LCC','T_salinity_LCC', 'T_lime_LCC', 'wtd_LCC', 'T_perm_LCC', 
'T_LAWC','S_erosion_risk_LCC', 'S_soil_tex_LCC', 'S_salinity_LCC', 'S_lime_LCC','S_LAWC', 'S_perm_LCC', 
'Total_LAWC_LCC', 'LCC_final_T', 'LCC_final_S','T_limitation', 'S_limitation']].copy()
df116_succinct = df116[['AWC_CLASS','T_GRAVEL', 'T_SAND', 'T_SILT', 'T_CLAY', 'T_OC', 'T_PH_H2O','T_ECE', 'S_GRAVEL', 'S_SAND',
'S_SILT', 'S_CLAY', 'S_OC','S_PH_H2O',  'S_ECE', 'slope', 'T_AWC', 'S_AWC','T_erosion_risk_LCC', 
'T_soil_tex_LCC', 'Total_awc_LCC', 'depth_LCC','T_salinity_LCC', 'T_lime_LCC', 'wtd_LCC', 'T_perm_LCC', 
'T_LAWC','S_erosion_risk_LCC', 'S_soil_tex_LCC', 'S_salinity_LCC', 'S_lime_LCC','S_LAWC', 'S_perm_LCC', 
'Total_LAWC_LCC', 'LCC_final_T', 'LCC_final_S','T_limitation', 'S_limitation']].copy()

In [177]:
len(df69)

619509

In [178]:
df0_succinct.to_csv("LCC_HWSD_dosso0")
df69_succinct.to_csv("LCC_HWSD_dosso69")
df116_succinct.to_csv("LCC_HWSD_dosso116")